## [Resource](https://www.tensorflow.org/tutorials/quickstart/advanced)

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras import Model


tf.__version__

'2.3.1'

In [2]:
# Params List
BATCH_SIZE = 64
SHUFFLE_NUM = 10000
EPOCHS = 5

In [3]:
mnist_dataset = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist_dataset.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
# Add a channel dimension

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [5]:
# Shuffle op

train_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)
).shuffle(SHUFFLE_NUM).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

In [6]:
class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = Conv2D(BATCH_SIZE, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [7]:
model = CustomModel()

In [8]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [9]:
train_loss = tf.keras.metrics.Mean(name='Train_LOSS')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='Train_ACC')

test_loss = tf.keras.metrics.Mean(name='Test_LOSS')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='Test_ACC')


In [10]:
# GradientTape Train Model
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_obj(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [11]:
# GradientTap Testing Model
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_obj(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [12]:
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_dataset:
        train_step(images, labels)
        
    for test_img, test_lab in test_dataset:
        test_step(test_img, test_lab)
        
    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result()}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result()}'
  )

Epoch 1, Loss: 0.13924214243888855, Accuracy: 0.9587000012397766, Test Loss: 0.06102897971868515, Test Accuracy: 0.9800000190734863
Epoch 2, Loss: 0.042054641991853714, Accuracy: 0.9874333143234253, Test Loss: 0.050323981791734695, Test Accuracy: 0.9829999804496765
Epoch 3, Loss: 0.02025645039975643, Accuracy: 0.9933833479881287, Test Loss: 0.05020831525325775, Test Accuracy: 0.9840999841690063
Epoch 4, Loss: 0.012853105552494526, Accuracy: 0.9956833124160767, Test Loss: 0.0553613118827343, Test Accuracy: 0.9843999743461609
Epoch 5, Loss: 0.009503312408924103, Accuracy: 0.9969000220298767, Test Loss: 0.07147934287786484, Test Accuracy: 0.9811000227928162


In [13]:
model.summary()

Model: "custom_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  640       
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  5537920   
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 5,539,850
Trainable params: 5,539,850
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.save_weights('model/mnist_99_acc')